<a href="https://colab.research.google.com/github/kavyajeetbora/foursquare_ai/blob/master/notebooks/16%20-%20Generate%20tiles%20db%20from%20overture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade duckdb

In [ ]:
import duckdb
print(duckdb.__version__)          # should be 1.1.x or newer in 2026

con = duckdb.connect()           # or duckdb.connect(':memory:')

con.sql("INSTALL spatial;")
con.sql("INSTALL httpfs;")       # almost always needed for s3:// paths

con.sql("LOAD spatial;")
con.sql("LOAD httpfs;")

In [ ]:
con = duckdb.connect('tiles.db')  # or choose a new name e.g. 'nyc_buildings.db' to avoid confusion

con.sql("INSTALL spatial; LOAD spatial;")   # make sure extension is ready

con.sql("""
CREATE OR REPLACE TABLE t1 AS (
  SELECT
    ST_Transform(geometry, 'EPSG:4326', 'EPSG:3857', always_xy := true) AS geometry,
    subtype,
    class,
    height
  FROM read_parquet(
    's3://overturemaps-us-west-2/release/2026-01-21.0/theme=buildings/type=building/*',
    filename = true,
    hive_partitioning = 1
  )
  WHERE
    bbox.xmin BETWEEN -74.2 AND -73.6
    AND bbox.ymin BETWEEN 40.5  AND 40.9
    AND bbox.xmax BETWEEN -74.2 AND -73.6
    AND bbox.ymax BETWEEN 40.5  AND 40.9
    AND subtype IS NOT NULL
    AND class   IS NOT NULL
    AND height  IS NOT NULL
);
""")

con.sql("CREATE INDEX my_idx ON t1 USING RTREE (geometry);")

# Quick reality check — should print a number > 0 if data loaded
con.sql("SELECT COUNT(*) FROM t1").show()

# Optional: force write to disk and close cleanly
con.sql("CHECKPOINT")
con.close()

In [ ]:
!ls -lh tiles.db

In [ ]:
# 1. Basic smoke test – does DuckDB even see the file as non-empty?
con = duckdb.connect('tiles.db')
print(con.sql("SELECT * FROM duckdb_tables()").fetchall())          # lists user tables
print(con.sql("SELECT * FROM duckdb_databases()").fetchall())       # should show main db
print(con.sql("PRAGMA database_size").fetchall())                   # rough size info

# 2. Also check for any leftover attached databases
con.sql("SHOW DATABASES").show()

In [ ]:
con = duckdb.connect('tiles.db')   # creates file in Colab's /content/
# then run the same INSTALL / LOAD commands
# then your original code with attach/use/detach

con.sql("SELECT COUNT(*) FROM t1").show()

con.sql("""
SELECT
  subtype,
  COUNT(*) AS cnt,
  AVG(height) AS avg_height
FROM t1
GROUP BY 1
ORDER BY cnt DESC
LIMIT 10
""").show()